# Preprocessing the data
## Target
- input: mimic_iv csv files
    - cohort.csv
    - ground_truth.csv
    - baseline.csv
    - vitalsign_gcs.csv
    - vitalsign.csv
    - ventilator_settings.csv
- output: three csv files
    - baseline_charttime_ground_truth.csv: variable length for each patients trajectory
    - baseline_charttime_ground_truth_n_hr.csv: fixed length (e.g., 1hr(last), 24hr, or 48hr) for each patients trajectory
    - baseline_charttime_ground_truth_mode_change.csv:
        - ventilator_mode_group_change: change from Minimal Support to Complete Support set as -1, Complete Support to Minimal Support set as 1
    - columns
        - id_col: ['stay_id', 'before_weaning_hr']
        - state_variables_col = ['age', 'gender', 'race', 'weight_kg', 'height_cm', 'tobacco', 'tidal_volume_observed', 'RSBI', 'minute_ventilation', 'heart_rate', 'sbp', 'dbp', 'mbp', 'resp_rate', 'spo2', 'gcs']
        - action_variables_col = ['peep', 'fio2', 'respiratory_rate_set', 'tidal_volume_set', 'plateau_pressure', 'ventilator_mode_group']
        - outcome_col = ['weaning_till_reintubation_hr', 'weaning_till_dod_hr']
    - rows
        - fixed rows: 7093 * 24 --> 6798 * 24 (after drop having whole miss features)
        - variable rows: 7093 * each trajectroy length --> 6798 * each trajectroy length
## Baseline Steps
- category to onehot: gender and race
## Charttime Steps
### Generate template of each trajectory
- category to numerical: ventilator_mode_group 0 minimal, 1 partial, 2 complete
- stay_id and before_weaning_hr, can get extra 4 hr for fill missing value before ventilation starttime
- fixed length: 4 + 24 = 28 rows for each trajectroy
- variable length: 4 + trajectory length rows
    - based on cohort.csv, calculate the length of endtime and start time, then generate a dictionary: stay_id -> vent_hr
- Do the second version, more general for two
## Remove outliers
- Based on Tukey's fences for outlier detection, which are defines 'non outlier' data as everything in the range ``[(first quartile - 1.5 IQR), (third quartile + 1.5 IQR)]
- Based on textbook and paper, we get the reasonable range for each variables
- we choose the looser one as bound to exclude the outliers 
## Fill missing value
- KNN for baseline
- ffill and bfill for charttime

## Read in data
- cohort.csv: subject_id, stay_id, starttime, endtime (start/end time of ventilation event)
- ground_truth.csv: subject_id, stay_id, starttime, endtime, reintubation_time_gap_hr, dod
- baseline.csv: gender, race, age, weight, height, tobacco
- vitalsign_gcs.csv: subject_id, stay_id, charttime, gcs
- vitalsign.csv: subject_id, stay_id, charttime, heart_rate, resp_rate, sbp, dbp, mbp, spo2
- ventilator_settings.csv: subject_id, stay_id, charttime, peep, fio2, tidal_volume_observed, tidal_volume_set, respiratory_rate_set, plateau_pressure, ventilator_mode

In [ ]:
import os
import numpy as np
import pandas as pd
import copy
pd.set_option('display.max_columns', None)


In [ ]:
first_time_flag = True
DATA_SOURCE = "MIMIC-III" # "MIMIC-III", "eICU", "MIMIC-IV"
if DATA_SOURCE == "eICU":
    selected_columns = ['peep', 'respiratory_rate_set', 'fio2', 'heart_rate', 'resp_rate', 'spo2', 'sbp', 'dbp', 'mbp'] # TODO
else:
    selected_columns = ['peep', 'respiratory_rate_set', 'fio2', 'tidal_volume_set', 'heart_rate', 'resp_rate', 'spo2', 'sbp', 'dbp', 'mbp', 'tidal_volume_observed']

In [ ]:
prefix_path = "../data/mimic_iii"
# prefix_path = "../data/eICU"
# prefix_path = "../data/mimic_iv"

In [ ]:
os.listdir(prefix_path)

In [ ]:
baseline_df = pd.read_csv(f'{prefix_path}/baseline.csv')
cohort_df = pd.read_csv(f'{prefix_path}/cohort.csv')
ground_truth_df = pd.read_csv(f'{prefix_path}/ground_truth.csv')
ventilator_settings_df = pd.read_csv(f'{prefix_path}/ventilator_settings.csv')
vitalsign_df = pd.read_csv(f'{prefix_path}/vitalsign.csv')
vitalsign_gcs_df = pd.read_csv(f'{prefix_path}/vitalsign_gcs.csv')
if DATA_SOURCE != "eICU":
    ground_truth_deathtime_df = pd.read_csv(f'{prefix_path}/ground_truth_deathtime.csv')

In [ ]:
# Function to convert and format timestamps
def convert_timestamps(df, columns):
    for col in columns:
        # Convert to datetime
        df[col] = pd.to_datetime(df[col], errors='coerce')
        
        # Format datetime to the desired format
        df[col] = df[col].dt.strftime('%Y-%m-%d %H:%M:%S')
    
    return df

def convert_minutes_to_datetime(df, column='charttime'):
    # Make a copy of the dataframe to avoid modifying the original
    df = df.copy()
    
    # Convert minutes to seconds (multiply by 60)
    # Then convert seconds to datetime
    df[column] = pd.to_datetime(df[column].astype(float) * 60, unit='s')
    
    return df

if DATA_SOURCE == "eICU":
    # Rename 'patientunitstayid' to 'stay_id' in each DataFrame
    baseline_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)
    cohort_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)
    ground_truth_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)
    ventilator_settings_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)
    vitalsign_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)
    vitalsign_gcs_df.rename(columns={'patientunitstayid': 'stay_id'}, inplace=True)

    # Convert gender values from "Female"/"Male" to "F"/"M"
    baseline_df['gender'] = baseline_df['gender'].replace({'Female': 'F', 'Male': 'M'})

    # List of columns to convert
    timestamp_columns = ['starttime', 'endtime']

    # Convert timestamps in each DataFrame
    baseline_df = convert_timestamps(baseline_df, timestamp_columns)
    cohort_df = convert_timestamps(cohort_df, timestamp_columns)
    ground_truth_df = convert_timestamps(ground_truth_df, timestamp_columns)
    # Apply the conversion to each DataFrame
    ventilator_settings_df = convert_minutes_to_datetime(ventilator_settings_df)
    vitalsign_df = convert_minutes_to_datetime(vitalsign_df)
    vitalsign_gcs_df = convert_minutes_to_datetime(vitalsign_gcs_df)

    # Using the insert() method to add column at position 0 (first position)
    baseline_df.insert(0, "subject_id", baseline_df["stay_id"])
    cohort_df.insert(0, "subject_id", cohort_df["stay_id"])
    ground_truth_df.insert(0, "subject_id", ground_truth_df["stay_id"])
    ventilator_settings_df.insert(0, "subject_id", ventilator_settings_df["stay_id"])
    vitalsign_df.insert(0, "subject_id", vitalsign_df["stay_id"])
    vitalsign_gcs_df.insert(0, "subject_id", vitalsign_gcs_df["stay_id"])

    # Calculate the difference in hours
    baseline_df['time_diff'] = (pd.to_datetime(baseline_df['endtime']) - pd.to_datetime(baseline_df['starttime'])).dt.total_seconds() / 3600
    cohort_df['time_diff'] = (pd.to_datetime(cohort_df['endtime']) - pd.to_datetime(cohort_df['starttime'])).dt.total_seconds() / 3600
    ground_truth_df['time_diff'] = (pd.to_datetime(ground_truth_df['endtime']) - pd.to_datetime(ground_truth_df['starttime'])).dt.total_seconds() / 3600

    # Drop rows where the difference is greater than 1500 hours
    baseline_df = baseline_df[baseline_df['time_diff'] <= 1500]
    cohort_df = cohort_df[cohort_df['time_diff'] <= 1500]
    ground_truth_df = ground_truth_df[ground_truth_df['time_diff'] <= 1500]

    # Drop the time_diff column as it's no longer needed
    baseline_df = baseline_df.drop(columns=['time_diff'])
    cohort_df = cohort_df.drop(columns=['time_diff'])
    ground_truth_df = ground_truth_df.drop(columns=['time_diff'])

In [ ]:
if DATA_SOURCE == "MIMIC-III":
    # Rename 'icustay_id' to 'stay_id' in each DataFrame
    baseline_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    cohort_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    ground_truth_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    ground_truth_deathtime_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    ventilator_settings_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    vitalsign_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)
    vitalsign_gcs_df.rename(columns={'icustay_id': 'stay_id'}, inplace=True)

In [ ]:
# baseline_df['height'].isna().sum()
vitalsign_df.head()

In [ ]:
baseline_df.head()


In [ ]:
ventilator_settings_df.head()

In [ ]:
if DATA_SOURCE == "eICU":
    # there are 51 patients in eICU don't have any vital sign data
    print(len(cohort_df.join(vitalsign_df.groupby(['stay_id', 'subject_id']).count(), how='left', on=['stay_id', 'subject_id'])[cohort_df.join(vitalsign_df.groupby(['stay_id', 'subject_id']).count(), how='left', on=['stay_id', 'subject_id'])["heart_rate"].isna()]))

## Baseline preprocessing

In [ ]:
baseline_df

In [ ]:
baseline_df.groupby("race").count()

### Category to one-hot

#### Race mapping

In [ ]:
def group_ethnicity(ethnicity):
    # Define the mappings
    white_group = ['WHITE', 'WHITE - OTHER EUROPEAN', 'WHITE - RUSSIAN', 'WHITE - BRAZILIAN', 'WHITE - EASTERN EUROPEAN', 'Caucasian']
    asian_group = ['ASIAN', 'ASIAN - CHINESE', 'ASIAN - SOUTH EAST ASIAN', 'ASIAN - ASIAN INDIAN', 'ASIAN - KOREAN', 'Asian']
    black_group = ['BLACK/AFRICAN AMERICAN', 'BLACK/CAPE VERDEAN', 'BLACK/CARIBBEAN ISLAND', 'BLACK/AFRICAN', 'African American']
    hispanic_group = ['HISPANIC/LATINO - PUERTO RICAN', 'HISPANIC OR LATINO', 'HISPANIC/LATINO - COLUMBIAN', 
                      'HISPANIC/LATINO - DOMINICAN', 'HISPANIC/LATINO - HONDURAN', 'HISPANIC/LATINO - CENTRAL AMERICAN', 
                      'HISPANIC/LATINO - GUATEMALAN', 'HISPANIC/LATINO - MEXICAN', 'HISPANIC/LATINO - SALVADORAN', 'Hispanic']
    others_group = ['UNKNOWN', 'PATIENT DECLINED TO ANSWER', 'OTHER', 'UNABLE TO OBTAIN', 'PORTUGUESE', 
                    'NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'AMERICAN INDIAN/ALASKA NATIVE', 
                    'MULTIPLE RACE/ETHNICITY', 'Native American', 'Other/Unknown']

    # Categorize based on the mappings
    if ethnicity in white_group:
        return 'WHITE'
    elif ethnicity in asian_group:
        return 'ASIAN'
    elif ethnicity in black_group:
        return 'BLACK'
    elif ethnicity in hispanic_group:
        return 'HISPANIC'
    else:
        return 'OTHERS'

In [ ]:
baseline_preprocess_df = baseline_df
baseline_preprocess_df['race_grouped'] = baseline_preprocess_df['race'].apply(group_ethnicity)
# One-hot encode the race_grouped column
baseline_preprocess_df = pd.get_dummies(baseline_preprocess_df, columns=['race_grouped'])
baseline_preprocess_df = baseline_preprocess_df.drop(['race'], axis=1)
baseline_preprocess_df

note: there are some stay_id have multiple race, choose one if not UNKNOWN or OTHERS

In [ ]:
# Define priority order for race groups
race_priority = ['race_grouped_ASIAN', 'race_grouped_BLACK', 'race_grouped_HISPANIC', 'race_grouped_WHITE', 'race_grouped_OTHERS']

# Sort the DataFrame based on the priority order and `stay_id`
baseline_preprocess_df['priority'] = baseline_preprocess_df[race_priority].idxmax(axis=1).apply(lambda x: race_priority.index(x))
baseline_preprocess_df = baseline_preprocess_df.sort_values(by=['stay_id', 'priority']).drop(columns='priority')

# Drop duplicate rows for each `stay_id`, keeping the first occurrence
baseline_preprocess_df = baseline_preprocess_df.drop_duplicates(subset=['stay_id'], keep='first')

In [ ]:
baseline_preprocess_df

In [ ]:
len(baseline_preprocess_df['stay_id'].unique())

#### Gender mapping

In [ ]:
baseline_preprocess_df = pd.get_dummies(baseline_preprocess_df, columns=['gender'])
baseline_preprocess_df

In [ ]:
if first_time_flag:
    baseline_preprocess_df.to_csv(f"{prefix_path}/baseline_preprocess.csv", index=False)

## Charttime preprocessing

#### generate template: stay_id and before_weaning_hr
- add extra 4 hr for fill missing value

In [ ]:
EXTRA_4_HR_FLAG = False

In [ ]:
cohort_hr_df = cohort_df
# Convert starttime and endtime to datetime
cohort_hr_df['starttime'] = pd.to_datetime(cohort_hr_df['starttime'])
cohort_hr_df['endtime'] = pd.to_datetime(cohort_hr_df['endtime'])

# Calculate intubation_hr
cohort_hr_df['intubation_hr'] = (cohort_hr_df['endtime'] - cohort_hr_df['starttime']).dt.total_seconds() / 3600
if EXTRA_4_HR_FLAG:
    cohort_hr_df['intubation_hr_add_4hr'] = cohort_hr_df['intubation_hr'] + 4
cohort_hr_df

In [ ]:
def generate_before_weaning_rows(stay_id, hours):
    rows = []
    for hr in range(int(hours), -1, -1):
        rows.append({'stay_id': stay_id, 'before_weaning_hr': hr})
    return rows

# Create the charttime_template_df
charttime_template_df = pd.DataFrame()

for index, row in cohort_hr_df.iterrows():
    stay_id = row['stay_id']
    if EXTRA_4_HR_FLAG:
        hours = row['intubation_hr_add_4hr']
    else:
        hours = row['intubation_hr']
    rows = generate_before_weaning_rows(stay_id, hours)
    charttime_template_df = pd.concat([charttime_template_df, pd.DataFrame(rows)], ignore_index=True)
charttime_template_df

### ventilator mode group mapping

In [ ]:
def categorize_ventilator_mode(ventilator_mode):
    complete_support = ["PRVC/AC", "PCV+Assist", "PCV+", "MMV/AutoFlow", "APRV", "CMV/AutoFlow",
                        "CMV", "PRES/AC (PCAC)", "PRES/AC", "APV (cmv)", "PRVC/SIMV (=aprv)", "PRVC/SIMV", "MMV",
                        "VOL/AC", "APRV/Biphasic+ApnVol", "APRV/Biphasic+ApnPress", "(S) CMV",
                        "P-CMV", "CMV/ASSIST", "MMV/PSV/AutoFlow", "CMV/ASSIST/AutoFlow"]

    partial_support = ["SIMV/PSV/AutoFlow", "SIMV/PRES", "SIMV/PSV", "SIMV/AutoFlow", "SIMV/VOL",
                       "SIMV", "SYNCHRON MASTER", "SYNCHRON SLAVE"]

    minimal_support = ["CPAP/PSV+ApnVol", "CPAP/PSV+Apn TCPL", "CPAP/PPS", "PCV+/PSV", "Apnea Ventilation", "CPAP",
                       "MMV/PSV", "SPONT", "CPAP/PSV+ApnPres", "Ambient", "CPAP/PSV+Apn TCPL(time cycle pressure limit)",
                       "null", "PSV/SBT", "Standby", "CPAP/PSV"]

    if ventilator_mode in complete_support:
        return "Complete Support"
    elif ventilator_mode in partial_support:
        return "Partial Support"
    elif ventilator_mode in minimal_support:
        return "Minimal Support"
    else:
        return "NaN"

In [ ]:
ventilator_settings_df['ventilator_mode_group'] = ventilator_settings_df['ventilator_mode'].apply(categorize_ventilator_mode)

### ventilator_settings before_weaning_hr

In [ ]:
ventilator_settings_hr_df = pd.merge(cohort_df, ventilator_settings_df, on=['stay_id', 'subject_id'])
ventilator_settings_hr_df['charttime'] = pd.to_datetime(ventilator_settings_hr_df['charttime'])
ventilator_settings_hr_df['endtime'] = pd.to_datetime(ventilator_settings_hr_df['endtime'])
ventilator_settings_hr_df['starttime'] = pd.to_datetime(ventilator_settings_hr_df['starttime'])
ventilator_settings_hr_df['before_weaning_hr'] = ((ventilator_settings_hr_df['endtime'] - ventilator_settings_hr_df['charttime']).dt.total_seconds() / 3600).astype(int)
ventilator_settings_hr_df

### merge ventilator settings charttime evnets into template

In [ ]:
# Perform a left join on stay_id and before_weaning_hr
charttime_template_vese_df = pd.merge(charttime_template_df, ventilator_settings_hr_df.drop(columns=['charttime']), on=['stay_id', 'before_weaning_hr'], how='left')
charttime_template_vese_df
# Filter rows where before_weaning_hr from ventilator_settings_hr_df does not appear in charttime_template_df
# filtered_df = charttime_template_vese_df.dropna(subset=['subject_id'])

In [ ]:
charttime_template_vese_df.head()

### vitalsign before_weaning_hr

In [ ]:
vitalsign_hr_df = pd.merge(cohort_df, vitalsign_df, on=['stay_id', 'subject_id'])
vitalsign_hr_df['charttime'] = pd.to_datetime(vitalsign_hr_df['charttime'])
vitalsign_hr_df['endtime'] = pd.to_datetime(vitalsign_hr_df['endtime'])
vitalsign_hr_df['starttime'] = pd.to_datetime(vitalsign_hr_df['starttime'])
vitalsign_hr_df['before_weaning_hr'] = ((vitalsign_hr_df['endtime'] - vitalsign_hr_df['charttime']).dt.total_seconds() / 3600).astype(int)
vitalsign_hr_df

In [ ]:
vitalsign_gcs_hr_df = pd.merge(cohort_df, vitalsign_gcs_df, on=['stay_id', 'subject_id'])
vitalsign_gcs_hr_df['charttime'] = pd.to_datetime(vitalsign_gcs_hr_df['charttime'])
vitalsign_gcs_hr_df['endtime'] = pd.to_datetime(vitalsign_gcs_hr_df['endtime'])
vitalsign_gcs_hr_df['starttime'] = pd.to_datetime(vitalsign_gcs_hr_df['starttime'])
vitalsign_gcs_hr_df['before_weaning_hr'] = ((vitalsign_gcs_hr_df['endtime'] - vitalsign_gcs_hr_df['charttime']).dt.total_seconds() / 3600).astype(int)
vitalsign_gcs_hr_df

### merge vitalsign charttime evnets into template

In [ ]:
# Perform a left join on stay_id and before_weaning_hr
if EXTRA_4_HR_FLAG:
    charttime_template_vese_visi_df = pd.merge(charttime_template_vese_df, vitalsign_hr_df.drop(columns=['charttime', 'starttime', 'endtime', 'intubation_hr', 'intubation_hr_add_4hr']), on=['stay_id', 'before_weaning_hr', 'subject_id'], how='left')
    charttime_template_vese_visi_df = pd.merge(charttime_template_vese_visi_df, vitalsign_gcs_hr_df.drop(columns=['charttime', 'starttime', 'endtime', 'intubation_hr', 'intubation_hr_add_4hr']), on=['stay_id', 'before_weaning_hr', 'subject_id'], how='left')
else:
    charttime_template_vese_visi_df = pd.merge(charttime_template_vese_df, vitalsign_hr_df.drop(columns=['charttime', 'starttime', 'endtime', 'intubation_hr']), on=['stay_id', 'before_weaning_hr', 'subject_id'], how='left')
    charttime_template_vese_visi_df = pd.merge(charttime_template_vese_visi_df, vitalsign_gcs_hr_df.drop(columns=['charttime', 'starttime', 'endtime', 'intubation_hr']), on=['stay_id', 'before_weaning_hr', 'subject_id'], how='left')
charttime_template_vese_visi_df

In [ ]:
charttime_template_vese_visi_df[80:140]

- add RSBI and minute_ventilation, need to use the resp_rate from vitalsign

In [ ]:
charttime_template_vese_visi_df['RSBI'] = charttime_template_vese_visi_df['resp_rate'] / (charttime_template_vese_visi_df['tidal_volume_observed'] * 0.001)
charttime_template_vese_visi_df['minute_ventilation'] = charttime_template_vese_visi_df['resp_rate'] * (charttime_template_vese_visi_df['tidal_volume_observed'] * 0.001)
charttime_template_vese_visi_df[:10]

In [ ]:
charttime_template_vese_visi_df[:60]

In [ ]:
charttime_template_vese_visi_df.columns

### pick max value for stay_id and before_weaning_hr have multiple rows
WARNING: this code cell run 92mins

In [ ]:
# def custom_agg(series):
#     non_nan_values = series.dropna().unique()
#     if len(non_nan_values) == 1:
#         return non_nan_values[0]
#     elif len(non_nan_values) > 1:
#         return non_nan_values.max()
#     else:
#         return np.nan

# # Group by 'stay_id' and 'before_weaning_hr' and apply the custom aggregation function
# if first_time_flag:
#     charttime_template_vese_visi_hr_df = charttime_template_vese_visi_df.groupby(['stay_id', 'before_weaning_hr']).agg(custom_agg).reset_index()
#     charttime_template_vese_visi_hr_df

#### pick most severe value for stay_id and before_weaning_hr have multiple rows


In [ ]:
import numpy as np

def agg_heart_rate(x):
    arr = x.dropna().values
    if len(arr) == 0:
        return np.nan
    below = arr[arr < 60]
    above = arr[arr > 130]
    if len(above) > 0:
        return above.max()
    elif len(below) > 0:
        return below.min()
    else:
        return np.median(arr)

def agg_resp_rate(x):
    arr = x.dropna().values
    if len(arr) == 0:
        return np.nan
    below = arr[arr < 12]
    above = arr[arr > 30]
    if len(above) > 0:
        return above.max()
    elif len(below) > 0:
        return below.min()
    else:
        return np.median(arr)

def agg_spo2(x):
    arr = x.dropna()
    arr = arr[arr > 0]
    if len(arr) == 0:
        return np.nan
    return arr.min()

def agg_max(x):
    arr = x.dropna()
    if len(arr) == 0:
        return np.nan
    return arr.max()

def agg_sbp(x):
    arr = x.dropna().values
    if len(arr) == 0:
        return np.nan
    below = arr[arr < 90]
    above = arr[arr > 160]
    if len(above) > 0:
        return above.max()
    elif len(below) > 0:
        return below.min()
    else:
        return np.median(arr)

def agg_mbp(x):
    arr = x.dropna().values
    if len(arr) == 0:
        return np.nan
    below = arr[arr < 70]
    above = arr[arr > 130]
    if len(above) > 0:
        return above.max()
    elif len(below) > 0:
        return below.min()
    else:
        return np.median(arr)

def agg_dbp(x):
    arr = x.dropna().values
    if len(arr) == 0:
        return np.nan
    below = arr[arr < 70]
    above = arr[arr > 130]
    if len(above) > 0:
        return above.max()
    elif len(below) > 0:
        return below.min()
    else:
        return np.median(arr)

def agg_min(x):
    arr = x.dropna()
    if len(arr) == 0:
        return np.nan
    return arr.min()

def agg_rsbi(x):
    arr = x.dropna()
    if len(arr) == 0:
        return np.nan
    return arr.max()

def agg_minute_ventilation(x):
    arr = x.dropna()
    if len(arr) == 0:
        return np.nan
    return arr.min()

# For columns not listed, use first non-null value
def agg_first(x):
    arr = x.dropna()
    if len(arr) == 0:
        return np.nan
    return arr.iloc[0]

agg_dict = {
    'subject_id': agg_first,
    'starttime': agg_first,
    'endtime': agg_first,
    'intubation_hr': agg_first,
    'peep': agg_max,
    'fio2': agg_max,
    'tidal_volume_observed': agg_max,
    'tidal_volume_set': agg_max,
    'respiratory_rate_set': agg_max,
    'plateau_pressure': agg_max,
    'ventilator_mode': agg_first,
    'ventilator_mode_group': agg_first,
    'heart_rate': agg_heart_rate,
    'resp_rate': agg_resp_rate,
    'sbp': agg_sbp,
    'dbp': agg_dbp,
    'mbp': agg_mbp,
    'spo2': agg_spo2,
    'gcs': agg_min,
    'RSBI': agg_rsbi,
    'minute_ventilation': agg_minute_ventilation,
}

if first_time_flag:
    charttime_template_vese_visi_hr_df = charttime_template_vese_visi_df.groupby(['stay_id', 'before_weaning_hr']).agg(agg_dict).reset_index()
    charttime_template_vese_visi_hr_df

In [ ]:
if first_time_flag:
    charttime_template_vese_visi_hr_df = charttime_template_vese_visi_hr_df.sort_values(by=['stay_id', 'before_weaning_hr'], ascending=[True, False])
    charttime_template_vese_visi_hr_df

In [ ]:
if first_time_flag:
    charttime_template_vese_visi_hr_df.to_csv(f"{prefix_path}/charttime_template_vese_visi_hr.csv")
else:
    charttime_template_vese_visi_hr_df = pd.read_csv(f"{prefix_path}/charttime_template_vese_visi_hr.csv")
charttime_template_vese_visi_hr_df

In [ ]:
charttime_template_vese_visi_hr_df

## Remove outliers
- baseline_preprocess_df: don't needed
- charttime_template_vese_visi_hr_df:
    - based on Tukey's fences for outlier detection, which are defines 'non outlier' data as everything in the range ``[(first quartile - 1.5 IQR), (third quartile + 1.5 IQR)]
    - based on textbook and paper, we get the reasonable range for each variables
    - we choose the looser one as bound to exclude the outliers

In [ ]:
charttime_template_vese_visi_hr_df

In [ ]:
charttime_template_vese_visi_hr_df.describe()

- [MIT reference](https://emergency-vent.mit.edu/clinical/key-ventilation-specifications/)

PEEP (Positive End-Expiratory Pressure): 5–20 cmH2O

Reference: "Mechanical Ventilation: Clinical Applications and Pathophysiology" by Arthur S. Slutsky and Laurent Brochard, which suggests a typical PEEP range of 5-20 cmH2O for mechanically ventilated patients.
FiO2 (Fraction of Inspired Oxygen): 21–100%

Reference: Standard clinical practice guidelines for mechanical ventilation typically suggest a FiO2 range of 21–100% to maintain adequate oxygenation.
Tidal Volume Observed and Tidal Volume Set: 200–800 mL

Reference: "Mechanical Ventilation" by David C. Shelledy and Jay I. Peters, which indicates that the typical range for tidal volume is between 6-8 mL/kg of ideal body weight, translating to approximately 200-800 mL for most adults.
Respiratory Rate Set: 6–40 breaths/min

Reference: "Pilbeam's Mechanical Ventilation: Physiological and Clinical Applications" by J.M. Cairo and D.C. Pilbeam, which mentions a typical range of 6–40 breaths/min for ventilator settings.
Plateau Pressure: 10–35 cmH2O

Reference: "The Principles and Practice of Mechanical Ventilation" by Martin J. Tobin, which states that plateau pressures should ideally be kept below 30-35 cmH2O to prevent ventilator-induced lung injury.
Heart Rate: 40–180 beats/min

Reference: "Harrison's Principles of Internal Medicine" by J. Larry Jameson, et al., which outlines normal and critical ranges for heart rates in clinical settings.
Respiratory Rate: 8–30 breaths/min

Reference: "Goldman-Cecil Medicine" by Lee Goldman and Andrew I. Schafer, which suggests typical clinical ranges for respiratory rates.
Systolic Blood Pressure (SBP): 70–200 mmHg

Reference: "Critical Care Medicine: Principles of Diagnosis and Management in the Adult" by Joseph E. Parrillo and R. Phillip Dellinger, which provides clinical ranges for blood pressure.
Diastolic Blood Pressure (DBP): 40–120 mmHg

Reference: Same as SBP reference above.
Mean Blood Pressure (MBP): 50–150 mmHg

Reference: Same as SBP reference above.
SpO2 (Oxygen Saturation): 80–100%

Reference: Clinical guidelines for oxygen therapy generally recommend maintaining SpO2 levels between 80-100%.
GCS (Glasgow Coma Scale): 3–15

Reference: Standard clinical use of the Glasgow Coma Scale, which ranges from 3 (deep unconsciousness) to 15 (fully alert).
RSBI (Rapid Shallow Breathing Index): 0–100

Reference: "Mechanical Ventilation" by David C. Shelledy and Jay I. Peters, which describes RSBI values used for weaning patients from mechanical ventilation.
Minute Ventilation: 2–20 L/min

Reference: "The Principles and Practice of Mechanical Ventilation" by Martin J. Tobin, which outlines typical minute ventilation ranges for adults.
These references provide the standard guidelines and ranges used in clinical practice to ensure patient safety and efficacy of mechanical ventilation settings.

In [ ]:
import numpy as np

# Function to calculate the bounds and set outliers to NaN
def set_outliers_to_nan(df, column, guidelines):
    Q1 = df[column].quantile(0.25)
    Q3 = df[column].quantile(0.75)
    IQR = Q3 - Q1
    data_lower_bound = Q1 - 1.5 * IQR
    data_upper_bound = Q3 + 1.5 * IQR
    guideline_lower_bound = guidelines[column]['lower']
    guideline_upper_bound = guidelines[column]['upper']
    
    # Determine the stricter bounds
    lower_bound = min(data_lower_bound, guideline_lower_bound)
    upper_bound = max(data_upper_bound, guideline_upper_bound)
    
    # Apply the bounds to the column, setting outliers to NaN
    df[column] = df[column].apply(lambda x: x if pd.notnull(x) and lower_bound <= x <= upper_bound else np.nan)

# Dictionary with guidelines for each variable
guidelines = {
    'peep': {'lower': 0, 'upper': 20},
    'fio2': {'lower': 21, 'upper': 100},
    'tidal_volume_observed': {'lower': 200, 'upper': 800},
    'tidal_volume_set': {'lower': 200, 'upper': 800},
    'respiratory_rate_set': {'lower': 6, 'upper': 40},
    'plateau_pressure': {'lower': 10, 'upper': 35},
    'heart_rate': {'lower': 40, 'upper': 180},
    'resp_rate': {'lower': 8, 'upper': 30},
    'sbp': {'lower': 70, 'upper': 200},
    'dbp': {'lower': 40, 'upper': 120},
    'mbp': {'lower': 50, 'upper': 150},
    'spo2': {'lower': 80, 'upper': 100},
    'gcs': {'lower': 3, 'upper': 15},
    'RSBI': {'lower': 0, 'upper': 100},
    'minute_ventilation': {'lower': 2, 'upper': 20}
}

# List of columns to check for outliers
columns_to_check = [
    'peep', 'fio2', 'tidal_volume_observed', 'tidal_volume_set',
    'respiratory_rate_set', 'plateau_pressure', 'heart_rate', 'resp_rate',
    'sbp', 'dbp', 'mbp', 'spo2', 'gcs', 'RSBI', 'minute_ventilation'
]

# Copy the dataframe
charttime_template_vese_visi_hr_without_outliers_df = charttime_template_vese_visi_hr_df.copy(deep=True)

# Apply the function to each column
for col in columns_to_check:
    set_outliers_to_nan(charttime_template_vese_visi_hr_without_outliers_df, col, guidelines)

charttime_template_vese_visi_hr_without_outliers_df


In [ ]:
charttime_template_vese_visi_hr_df.describe()

In [ ]:
charttime_template_vese_visi_hr_without_outliers_df.describe()

In [ ]:
charttime_template_vese_visi_hr_without_outliers_df['ventilator_mode_group'].unique()

In [ ]:
charttime_template_vese_visi_hr_without_outliers_df[["ventilator_mode_group"]].isna().mean()

In [ ]:
charttime_template_vese_visi_hr_without_outliers_df[:60]
# charttime_template_vese_visi_hr_without_outliers_df[60:100]

## Fill missing value
- baseline
- charttime

### Baseline
- weight & height -> KNN

In [ ]:
from sklearn.impute import KNNImputer

In [ ]:
baseline_preprocess_df.describe()

In [ ]:
baseline_fill_missing_df = baseline_preprocess_df.copy(deep=True)
# Select columns for imputation
columns_to_impute = ['age', 'weight', 'height', 'gender_F', 'gender_M']

# Separate the data to impute and the rest of the data
impute_data = baseline_fill_missing_df[columns_to_impute]

# Create KNN Imputer
imputer = KNNImputer(n_neighbors=3)

# Apply KNN Imputer
imputed_data = imputer.fit_transform(impute_data)

# Assign imputed values back to the DataFrame
baseline_fill_missing_df[columns_to_impute] = imputed_data

In [ ]:
baseline_fill_missing_df

### Charttime
- ffill then bfill

- first take a look at the miss distribution

In [ ]:
charttime_template_vese_visi_hr_without_outliers_df.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
# Calculate NaN percentage for each column
nan_percentage = charttime_template_vese_visi_hr_without_outliers_df.isna().mean() * 100

# Create a DataFrame to display the results
nan_analysis = pd.DataFrame({
    'Feature': nan_percentage.index,
    'NaN Count': charttime_template_vese_visi_hr_without_outliers_df.isna().sum(),
    'NaN Percentage': nan_percentage,
    'Non-NaN Count': charttime_template_vese_visi_hr_without_outliers_df.count(),
})

# Sort by NaN percentage (descending)
nan_analysis = nan_analysis.sort_values('NaN Percentage', ascending=False)

# Print the results
print("Missing Value Analysis:")
print(nan_analysis)

# Visualize NaN percentages
plt.figure(figsize=(12, 8))
plt.bar(nan_analysis['Feature'], nan_analysis['NaN Percentage'], color='skyblue')
plt.xticks(rotation=90)
plt.xlabel('Features')
plt.ylabel('NaN Percentage (%)')
plt.title('Percentage of Missing Values by Feature')
plt.axhline(y=50, color='r', linestyle='--', label='50% Missing')
plt.axhline(y=20, color='orange', linestyle='--', label='20% Missing')
plt.grid(axis='y', alpha=0.3)
plt.legend()
plt.tight_layout()
plt.show()

# Create a heatmap to visualize missing values
plt.figure(figsize=(12, 8))
sns.heatmap(charttime_template_vese_visi_hr_without_outliers_df.isna(), 
            cbar=False, 
            yticklabels=False,
            cmap='viridis')
plt.title('Missing Values Heatmap (Yellow = Missing)')
plt.tight_layout()
plt.show()

In [ ]:
# Fill missing values within each stay_id
charttime_template_vese_visi_hr_without_outliers_fill_missing_df = charttime_template_vese_visi_hr_without_outliers_df.copy(deep=True)
charttime_template_vese_visi_hr_without_outliers_fill_missing_df = charttime_template_vese_visi_hr_without_outliers_fill_missing_df.groupby('stay_id').apply(lambda group: group.ffill().bfill())
# Reset the index to avoid any issues caused by groupby
charttime_template_vese_visi_hr_without_outliers_fill_missing_df = charttime_template_vese_visi_hr_without_outliers_fill_missing_df.reset_index(drop=True)

In [ ]:
charttime_template_vese_visi_hr_without_outliers_count_mode_change_df = charttime_template_vese_visi_hr_without_outliers_fill_missing_df.copy(deep=True)
# Fill NaNs with forward fill and backward fill
target = "ventilator_mode_group"
# charttime_template_vese_visi_hr_without_outliers_count_mode_change_df[f'{target}'] = charttime_template_vese_visi_hr_without_outliers_count_mode_change_df.groupby('stay_id')[f'{target}'].ffill().bfill()

# Function to count changes in {target} within each stay_id
def count_changes(group):
    # group[f'{target}'] = group[f'{target}'].ffill().bfill()
    changes = (group[f'{target}'] != group[f'{target}'].shift(1)).sum() - 1
    return changes

# Group by stay_id and count changes
change_counts = charttime_template_vese_visi_hr_without_outliers_count_mode_change_df.groupby('stay_id').apply(count_changes)

# Calculate the average number of changes
average_changes = change_counts.mean()

# Display the result
print(f'Average number of changes in {target} per stay_id: {average_changes}')

- RSBI and minute_ventilation calculated by the filled result of resp_rate and tidal_volume_observed

In [ ]:
charttime_template_vese_visi_hr_without_outliers_fill_missing_df['RSBI'] = charttime_template_vese_visi_hr_without_outliers_fill_missing_df['resp_rate'] / (charttime_template_vese_visi_hr_without_outliers_fill_missing_df['tidal_volume_observed'] * 0.001)
charttime_template_vese_visi_hr_without_outliers_fill_missing_df['minute_ventilation'] = charttime_template_vese_visi_hr_without_outliers_fill_missing_df['resp_rate'] * (charttime_template_vese_visi_hr_without_outliers_fill_missing_df['tidal_volume_observed'] * 0.001)
charttime_template_vese_visi_hr_without_outliers_fill_missing_df[:10]

In [ ]:
charttime_template_vese_visi_hr_without_outliers_fill_missing_df.head()

## Generate ground truth label

In [ ]:
ground_truth_df

In [ ]:
if DATA_SOURCE == "eICU":
    # Create a deep copy of ground_truth_df
    ground_truth_with_label_df = ground_truth_df.copy(deep=True)
    
    # Add new columns with default values of 0
    ground_truth_with_label_df['dead_time_gap_hr'] = 0
    ground_truth_with_label_df['reintubation_time_gap_hr'] = 0
    
    # Create label column based on died_in_unit flag
    # 1 = successful weaning (patient didn't die in unit)
    # 0 = failed weaning (patient died in unit)
    ground_truth_with_label_df['label'] = ground_truth_with_label_df['died_in_unit'].apply(lambda x: 0 if x == True else 1)
    
    # Alternative formulation if you prefer:
    # ground_truth_with_label_df['label'] = (~ground_truth_with_label_df['died_in_unit']).astype(int)
    
    print("eICU data processing complete:")
    print(f"Label distribution: {ground_truth_with_label_df['label'].value_counts()}")
else:
    # Convert date columns to datetime
    ground_truth_with_label_df = ground_truth_df.copy(deep=True)
    ground_truth_with_label_df['endtime'] = pd.to_datetime(ground_truth_with_label_df['endtime'])
    ground_truth_with_label_df['dod'] = pd.to_datetime(ground_truth_with_label_df['dod'])

    # Calculate dead_time_gap_hr
    ground_truth_with_label_df['dead_time_gap_hr'] = (ground_truth_with_label_df['dod'] - ground_truth_with_label_df['endtime']).dt.total_seconds() / 3600

    # Generate label column
    conditions = [
        (ground_truth_with_label_df['reintubation_time_gap_hr'] < 48),
        (ground_truth_with_label_df['dead_time_gap_hr'] < 48)
    ]

    ground_truth_with_label_df['label'] = 1
    ground_truth_with_label_df.loc[conditions[0] | conditions[1], 'label'] = 0
ground_truth_with_label_df

## Combine baseline with charttime and ground truth
- baseline_fill_missing_df
- charttime_template_vese_visi_hr_without_outliers_fill_missing_df
- ground_truth_with_label_df

In [ ]:
if DATA_SOURCE == "eICU":
    baseline_fill_missing_df = baseline_fill_missing_df.drop(columns=['starttime', 'endtime'])
baseline_charttime_df = pd.merge(charttime_template_vese_visi_hr_without_outliers_fill_missing_df, baseline_fill_missing_df.drop(columns=['subject_id']), on=['stay_id'])
baseline_charttime_df

In [ ]:
baseline_charttime_ground_truth_df = pd.merge(baseline_charttime_df, ground_truth_with_label_df[["stay_id", "reintubation_time_gap_hr", "dead_time_gap_hr", "label"]], on=['stay_id'])
baseline_charttime_ground_truth_df

## Tableone

In [ ]:
from tableone import TableOne, load_dataset

In [ ]:
# Specify columns
if EXTRA_4_HR_FLAG:
    columns = [
        'stay_id', 'before_weaning_hr', 'subject_id', 'intubation_hr', 'intubation_hr_add_4hr', 
        'peep', 'fio2', 'tidal_volume_observed', 'tidal_volume_set', 'respiratory_rate_set', 
        'plateau_pressure', 'heart_rate', 'resp_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'gcs', 
        'RSBI', 'minute_ventilation', 'age', 'weight', 'height', 'tobacco', 'race_grouped_ASIAN', 
        'race_grouped_BLACK', 'race_grouped_HISPANIC', 'race_grouped_OTHERS', 'race_grouped_WHITE', 
        'gender_F', 'gender_M', 'reintubation_time_gap_hr', 'dead_time_gap_hr', 'label'
    ]
else:
    columns = [
        'stay_id', 'before_weaning_hr', 'subject_id', 'intubation_hr',
        'peep', 'fio2', 'tidal_volume_observed', 'tidal_volume_set', 'respiratory_rate_set', 
        'plateau_pressure', 'heart_rate', 'resp_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'gcs', 
        'RSBI', 'minute_ventilation', 'age', 'weight', 'height', 'tobacco', 'race_grouped_ASIAN', 
        'race_grouped_BLACK', 'race_grouped_HISPANIC', 'race_grouped_OTHERS', 'race_grouped_WHITE', 
        'gender_F', 'gender_M', 'reintubation_time_gap_hr', 'dead_time_gap_hr', 'label'
    ]

# Specify categorical columns
categorical = [
    'tobacco', 'race_grouped_ASIAN', 'race_grouped_BLACK', 'race_grouped_HISPANIC', 
    'race_grouped_OTHERS', 'race_grouped_WHITE', 'gender_F', 'gender_M', 'label'
]

# Create TableOne
if EXTRA_4_HR_FLAG:
    table = TableOne(baseline_charttime_ground_truth_df, columns=columns, categorical=categorical, nonnormal=['intubation_hr', 'intubation_hr_add_4hr'])
else:
    table = TableOne(baseline_charttime_ground_truth_df, columns=columns, categorical=categorical, nonnormal=['intubation_hr'])

# Display TableOne
print(table.tabulate(tablefmt="fancy_grid"))

In [ ]:
# Dropping rows with missing values in the specified columns
if DATA_SOURCE == "eICU" or DATA_SOURCE == "MIMIC-III":
    baseline_charttime_ground_truth_drop_missing_df = baseline_charttime_ground_truth_df.dropna(subset=selected_columns)
else:
    baseline_charttime_ground_truth_drop_missing_df = baseline_charttime_ground_truth_df.dropna(subset=selected_columns) # ['peep', 'tidal_volume_set', 'respiratory_rate_set', 'plateau_pressure', 'RSBI', 'gcs']
print(f"before: {len(baseline_charttime_ground_truth_df['stay_id'].unique())}")
print(f"after: {len(baseline_charttime_ground_truth_drop_missing_df['stay_id'].unique())}")
print(f"drop: {len(baseline_charttime_ground_truth_df['stay_id'].unique()) - len(baseline_charttime_ground_truth_drop_missing_df['stay_id'].unique())}")
print(f"ratio: {(len(baseline_charttime_ground_truth_df['stay_id'].unique()) - len(baseline_charttime_ground_truth_drop_missing_df['stay_id'].unique())) / len(baseline_charttime_ground_truth_df['stay_id'].unique())}")

In [ ]:
def analyze_feature_missing_rate(df, features):
    total_stays = len(df['stay_id'].unique())
    missing_counts = {}
    
    for feature in features:
        # Get stays where the feature is entirely missing
        feature_missing_stays = df.groupby('stay_id')[feature].apply(lambda x: x.isna().all())
        stays_missing_feature = feature_missing_stays[feature_missing_stays].index.tolist()
        
        missing_counts[feature] = {
            'num_stays_missing': len(stays_missing_feature),
            'pct_stays_missing': len(stays_missing_feature) / total_stays * 100
        }
    
    # Sort by missing rate (highest to lowest)
    sorted_features = sorted(missing_counts.items(), key=lambda x: x[1]['pct_stays_missing'], reverse=True)
    
    print(f"Total unique stay_ids: {total_stays}")
    print("\nMissing rates by feature (feature totally absent for entire stay):")
    print("-" * 70)
    print(f"{'Feature':<25} {'Stays Missing':<15} {'% of Total Stays':<20}")
    print("-" * 70)
    
    for feature, stats in sorted_features:
        print(f"{feature:<25} {stats['num_stays_missing']:<15} {stats['pct_stays_missing']:.2f}%")
    
    return missing_counts

In [ ]:
# Analyze which features are causing most of the drops
missing_rates = analyze_feature_missing_rate(baseline_charttime_ground_truth_df, selected_columns)

In [ ]:
# Specify columns
if EXTRA_4_HR_FLAG:
    columns = [
        'stay_id', 'before_weaning_hr', 'subject_id', 'intubation_hr', 'intubation_hr_add_4hr', 
        'peep', 'fio2', 'tidal_volume_observed', 'tidal_volume_set', 'respiratory_rate_set', 
        'plateau_pressure', 'heart_rate', 'resp_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'gcs', 
        'RSBI', 'minute_ventilation', 'age', 'weight', 'height', 'tobacco', 'race_grouped_ASIAN', 
        'race_grouped_BLACK', 'race_grouped_HISPANIC', 'race_grouped_OTHERS', 'race_grouped_WHITE', 
        'gender_F', 'gender_M', 'reintubation_time_gap_hr', 'dead_time_gap_hr', 'ventilator_mode_group', 'label'
    ]
else:
    columns = [
        'stay_id', 'before_weaning_hr', 'subject_id', 'intubation_hr', 
        'peep', 'fio2', 'tidal_volume_observed', 'tidal_volume_set', 'respiratory_rate_set', 
        'plateau_pressure', 'heart_rate', 'resp_rate', 'sbp', 'dbp', 'mbp', 'spo2', 'gcs', 
        'RSBI', 'minute_ventilation', 'age', 'weight', 'height', 'tobacco', 'race_grouped_ASIAN', 
        'race_grouped_BLACK', 'race_grouped_HISPANIC', 'race_grouped_OTHERS', 'race_grouped_WHITE', 
        'gender_F', 'gender_M', 'reintubation_time_gap_hr', 'dead_time_gap_hr', 'ventilator_mode_group', 'label'
    ]

# Specify categorical columns
categorical = [
    'tobacco', 'race_grouped_ASIAN', 'race_grouped_BLACK', 'race_grouped_HISPANIC', 
    'race_grouped_OTHERS', 'race_grouped_WHITE', 'gender_F', 'gender_M', 'ventilator_mode_group', 'label'
]

# Create TableOne
if EXTRA_4_HR_FLAG:
    table = TableOne(baseline_charttime_ground_truth_drop_missing_df, columns=columns, categorical=categorical, nonnormal=['intubation_hr', 'intubation_hr_add_4hr'])
else:
    table = TableOne(baseline_charttime_ground_truth_drop_missing_df, columns=columns, categorical=categorical, nonnormal=['intubation_hr', 'intubation_hr_add_4hr'])

# Display TableOne
print(table.tabulate(tablefmt="fancy_grid"))

### variable rows trajectroy
- baseline_charttime_ground_truth.csv

In [ ]:
len(baseline_charttime_ground_truth_drop_missing_df["stay_id"].unique())

In [ ]:
if first_time_flag:
    baseline_charttime_ground_truth_drop_missing_df.to_csv(f"{prefix_path}/baseline_charttime_ground_truth.csv", index=False)
    
    import json

    # Select the columns to normalize
    columns_to_normalize = baseline_charttime_ground_truth_drop_missing_df[["peep", "fio2", "tidal_volume_observed", "tidal_volume_set", "respiratory_rate_set", "plateau_pressure", "heart_rate", "resp_rate", "sbp", "dbp", "mbp", "spo2", "gcs", "RSBI", "minute_ventilation", "age", "weight", "height" ]]

    # Create a dictionary to store min and max values
    min_max_dict = {}

    # Calculate min and max for each column and store in the dictionary
    for col in columns_to_normalize:
        min_max_dict[col] = {
            'min': baseline_charttime_ground_truth_drop_missing_df[col].min(),
            'max': baseline_charttime_ground_truth_drop_missing_df[col].max()
        }

    # Save the dictionary to a file
    with open(f'{prefix_path}/min_max_values.json', 'w') as f:
        json.dump(min_max_dict, f)

    print("Min and max values saved successfully.")

## Preprocessing for ehrMGAN input format

## Update baseline_charttime_ground_truth.csv
- version 1: death_time_gap_hr is calculate from the mimic-iv-hospital.patient dod
- version 2: death_time_gap_hr is calculate from the mimic-iv-hospital.admission deathtime
    - ground_truth.csv -> ground_truth_deathtime.csv

In [ ]:
if DATA_SOURCE == "eICU":
    baseline_charttime_ground_truth_df = pd.read_csv(f"{prefix_path}/baseline_charttime_ground_truth.csv")
    ground_truth_deathtime_df = ground_truth_df.copy(deep=True)
    ground_truth_deathtime_df['deathtime'] = ground_truth_df['endtime']
elif DATA_SOURCE == "MIMIC-IV":
    baseline_charttime_ground_truth_df = pd.read_csv(f"{prefix_path}/baseline_charttime_ground_truth.csv")
    ground_truth_deathtime_df = pd.read_csv(f'{prefix_path}/ground_truth_deathtime.csv')

In [ ]:
baseline_charttime_ground_truth_df

In [ ]:
ground_truth_deathtime_df

In [ ]:
ground_truth_deathtime_df['endtime'] = pd.to_datetime(ground_truth_deathtime_df['endtime'])
ground_truth_deathtime_df['deathtime'] = pd.to_datetime(ground_truth_deathtime_df['deathtime'])

# Calculate dead_time_gap_hr
ground_truth_deathtime_df['dead_time_gap_hr'] = (ground_truth_deathtime_df['deathtime'] - ground_truth_deathtime_df['endtime']).dt.total_seconds() / 3600

# Display the result
ground_truth_deathtime_df[["stay_id", "deathtime", "dead_time_gap_hr"]]

In [ ]:
baseline_charttime_ground_truth_drop_dod_df = baseline_charttime_ground_truth_df.drop(columns=["dead_time_gap_hr"])
baseline_charttime_ground_truth_dead_time_df = baseline_charttime_ground_truth_drop_dod_df.merge(
    ground_truth_deathtime_df[["stay_id", "deathtime", "dead_time_gap_hr"]],
    on="stay_id",
    how="left"  # Use "left" join to keep all rows from the baseline DataFrame
)
baseline_charttime_ground_truth_dead_time_df

In [ ]:
baseline_charttime_ground_truth_dead_time_df[baseline_charttime_ground_truth_dead_time_df["dead_time_gap_hr"] < 0 ]

### Rule of label (new rule)
- dead:
    - a → 0	    dead before weaning: 74
    - b → -1	dead within 48hr after weaning: 925
    - c → 1	    no dead or reintubation within 48hr after weaning: 4691
- reintubation:
    - a → x	    null
    - b → -2	reintubation within 48hr after weaning: 467 (not sure used it or not)
    - c → 1	    same as above

In [ ]:
baseline_charttime_ground_truth_new_label_df = baseline_charttime_ground_truth_dead_time_df.drop(columns=["label"])

# Define conditions
conditions = [
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'] >= 48) & (baseline_charttime_ground_truth_new_label_df['reintubation_time_gap_hr'] >= 48),
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'] >= 48) & (baseline_charttime_ground_truth_new_label_df['reintubation_time_gap_hr'].isna()),
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'].isna()) & (baseline_charttime_ground_truth_new_label_df['reintubation_time_gap_hr'] >= 48),
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'].isna()) & (baseline_charttime_ground_truth_new_label_df['reintubation_time_gap_hr'].isna()),
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'] <= 0),
    (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'] > 0) & (baseline_charttime_ground_truth_new_label_df['dead_time_gap_hr'] < 48),
    (baseline_charttime_ground_truth_new_label_df['reintubation_time_gap_hr'] < 48),
]

# Define corresponding label values
choices = [1, 1, 1, 1, 0, -1, -2]

# Apply the conditions
baseline_charttime_ground_truth_new_label_df['label'] = np.select(conditions, choices, default=-3)
baseline_charttime_ground_truth_new_label_df

In [ ]:
baseline_charttime_ground_truth_new_label_df[baseline_charttime_ground_truth_new_label_df["label"] == -2].groupby("stay_id").count()

In [ ]:
unique_stay_id_df = baseline_charttime_ground_truth_new_label_df.drop_duplicates(subset='subject_id', keep='first')
# unique_stay_id_df = baseline_charttime_ground_truth_new_label_df.drop_duplicates(subset='stay_id', keep='first')
unique_stay_id_df.groupby("label").count()

In [ ]:
unique_stay_id_df.groupby("label").count()["stay_id"] / unique_stay_id_df.groupby("label").count()["stay_id"].sum()

In [ ]:
467+925+74+4691

In [ ]:
baseline_charttime_ground_truth_new_label_df

In [ ]:
if first_time_flag:
    baseline_charttime_ground_truth_new_label_df.to_csv(f"{prefix_path}/baseline_charttime_ground_truth_-2~1.csv", index=False)

### Label merge back to 0 and 1
- Due to the down stream task, we still need to map the label -2 ~ 1 to 0 / 1
- unique subject_id: 6157
- unique stay_id: 6798

In [ ]:
ground_truth_multi_stage_df = pd.read_csv(f'{prefix_path}/baseline_charttime_ground_truth_-2~1.csv')

In [ ]:
ground_truth_multi_stage_drop_df = ground_truth_multi_stage_df.drop_duplicates(subset=['subject_id'], keep='first')
ground_truth_multi_stage_drop_df.groupby("label").count()

In [ ]:
ground_truth_multi_stage_drop_df = ground_truth_multi_stage_df.drop_duplicates(subset=['stay_id'], keep='first')
ground_truth_multi_stage_drop_df.groupby("label").count()

In [ ]:
# Map label -1 to 0
ground_truth_multi_stage_df['label'] = ground_truth_multi_stage_df['label'].replace(-1, 0)
# Remove rows with label -2
ground_truth_multi_stage_without_reintubation_df = ground_truth_multi_stage_df[ground_truth_multi_stage_df['label'] != -2]

- rewrite the original version baseline_charttime_ground_truth.csv based on the reintubation / die as neg, otherwise pos, 6157 distinct subject_id

In [ ]:
ground_truth_multi_stage_df = pd.read_csv(f'{prefix_path}/baseline_charttime_ground_truth_-2~1.csv')
ground_truth_multi_stage_df

In [ ]:
ground_truth_multi_stage_df['label'] = ground_truth_multi_stage_df['label'].replace(-1, 0)
ground_truth_multi_stage_df['label'] = ground_truth_multi_stage_df['label'].replace(-2, 0)
# first_time_flag = False
if first_time_flag:
    ground_truth_multi_stage_df.to_csv(f'{prefix_path}/baseline_charttime_ground_truth_with_reintubation_binary_label.csv')

In [ ]:
ground_truth_multi_stage_df.drop_duplicates(subset='subject_id', keep='first').groupby("label").count()

In [ ]:
first_time_flag = False
if first_time_flag:
    ground_truth_multi_stage_without_reintubation_df.to_csv(f"{prefix_path}/baseline_charttime_ground_truth.csv", index=False)
first_time_flag = True

## Generate three csv files
- fixed_rows_trajectory.csv: fixed length (e.g., 24hr, or 48hr) for each patients trajectory
- var_rows_trajectory.csv: variable length for each patients trajectory
- last_state_with_outcome.csv: last state variables with ground truth outcome

In [ ]:
baseline_charttime_ground_truth_drop_missing_df = pd.read_csv(f"{prefix_path}/baseline_charttime_ground_truth.csv")
baseline_charttime_ground_truth_drop_missing_df

### fixed rows trajectory:
- baseline_charttime_ground_truth_0_hr.csv
- baseline_charttime_ground_truth_24_hr.csv
- baseline_charttime_ground_truth_48_hr.csv

In [ ]:
before_weaning_n_hr_list = [1, 24, 48]

# Filter the DataFrame
if first_time_flag:
    for n_hr in before_weaning_n_hr_list:
        baseline_charttime_ground_truth_n_hr_df = baseline_charttime_ground_truth_drop_missing_df[baseline_charttime_ground_truth_drop_missing_df['before_weaning_hr'] < n_hr]
        baseline_charttime_ground_truth_n_hr_df.to_csv(f"{prefix_path}/baseline_charttime_ground_truth_{n_hr}_hr.csv", index=False)

In [ ]:
baseline_charttime_ground_truth_n_hr_df

In [ ]:
48 * 6798

### ventilator_mode_group_change_transition.csv

In [ ]:
# Initialize the new column with default value 0
baseline_charttime_ground_truth_mode_change_df = baseline_charttime_ground_truth_df.copy(deep=True)
baseline_charttime_ground_truth_mode_change_df['ventilator_mode_group_change'] = 0

# Iterate over the DataFrame to set the ventilator_mode_group_change values
for i in range(len(baseline_charttime_ground_truth_mode_change_df) - 1):
    if baseline_charttime_ground_truth_mode_change_df.loc[i, 'stay_id'] == baseline_charttime_ground_truth_mode_change_df.loc[i + 1, 'stay_id']:
        current_mode = baseline_charttime_ground_truth_mode_change_df.loc[i, 'ventilator_mode_group']
        next_mode = baseline_charttime_ground_truth_mode_change_df.loc[i + 1, 'ventilator_mode_group']
        
        if current_mode == 'Minimal Support' and next_mode == 'Complete Support':
            baseline_charttime_ground_truth_mode_change_df.at[i, 'ventilator_mode_group_change'] = -1
        elif current_mode == 'Complete Support' and next_mode == 'Minimal Support':
            baseline_charttime_ground_truth_mode_change_df.at[i, 'ventilator_mode_group_change'] = 1

In [ ]:
baseline_charttime_ground_truth_mode_change_subset_df = baseline_charttime_ground_truth_mode_change_df[baseline_charttime_ground_truth_mode_change_df["ventilator_mode_group_change"] != 0]
baseline_charttime_ground_truth_mode_change_subset_df

change ratio: around 2% transitions change ventilator mode group

In [ ]:
len(baseline_charttime_ground_truth_mode_change_subset_df) / len(baseline_charttime_ground_truth_mode_change_df)

In [ ]:
worse_count = len(baseline_charttime_ground_truth_mode_change_df[baseline_charttime_ground_truth_mode_change_df["ventilator_mode_group_change"] == -1])
better_count = len(baseline_charttime_ground_truth_mode_change_df[baseline_charttime_ground_truth_mode_change_df["ventilator_mode_group_change"] == 1])
print(f'ventilator mode group change worse: {worse_count}')
print(f'ventilator mode group change better: {better_count}')
print(f"{worse_count/(worse_count + better_count)} : {better_count/(worse_count + better_count)}")

In [ ]:
baseline_charttime_ground_truth_mode_change_subset_drop_missing_df = baseline_charttime_ground_truth_mode_change_subset_df.dropna(subset=['peep', 'tidal_volume_set', 'respiratory_rate_set', 'plateau_pressure', 'RSBI'])

In [ ]:
if first_time_flag:
    baseline_charttime_ground_truth_mode_change_subset_drop_missing_df.to_csv(f"{prefix_path}/baseline_charttime_ground_truth_mode_change.csv", index=False)

### TODO: for those ventilator settings mode change, maybe single hr transition is not enough, since it won't consider how "stable" of the patient state

## Analysis for Cohort Description

In [ ]:
import os
import json
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter, defaultdict
from sklearn.manifold import TSNE

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)
pd.set_option('display.max_columns', None)

In [ ]:
# ================================
# Data reading
# ================================
# Define data paths (update if needed)
eICU_prefix_path = "../data/eICU"
mimic_iii_prefix_path = "../data/mimic_iii"
mimic_iv_prefix_path = "../data/mimic_iv"

eICU_file = os.path.join(eICU_prefix_path, "baseline_charttime_ground_truth.csv")
mimic_iv_file = os.path.join(mimic_iv_prefix_path, "baseline_charttime_ground_truth.csv")
mimic_iii_file = os.path.join(mimic_iii_prefix_path, "baseline_charttime_ground_truth.csv")

# Read dataframes
eICU_df = pd.read_csv(eICU_file)
mimic_iv_df = pd.read_csv(mimic_iv_file)
mimic_iii_df = pd.read_csv(mimic_iii_file)

In [ ]:
mimic_iv_df

In [ ]:
eICU_df

In [ ]:
import pandas as pd
import numpy as np

def cohort_description(df, dataset_name):
    """
    Generate cohort description for race, sepsis, ARDS, gender, and age.
    
    Parameters:
    df (pandas.DataFrame): Input DataFrame (MIMIC-IV or eICU)
    dataset_name (str): Name of the dataset ('MIMIC-IV' or 'eICU')
    
    Returns:
    dict: Summary statistics for the cohort
    """
    # Initialize results dictionary
    results = {'Dataset': dataset_name}
    
    # Total number of stays
    total_stays = df['stay_id'].nunique()
    results['Total Stays'] = total_stays
    
    # Race distribution
    race_columns = ['race_grouped_ASIAN', 'race_grouped_BLACK', 'race_grouped_HISPANIC', 
                    'race_grouped_OTHERS', 'race_grouped_WHITE']
    race_counts = {}
    for col in race_columns:
        race_name = col.split('_')[-1].capitalize()
        count = df[df[col] == True]['stay_id'].nunique()
        percentage = (count / total_stays * 100) if total_stays > 0 else 0
        race_counts[race_name] = {'Count': count, 'Percentage': round(percentage, 2)}
    results['Race Distribution'] = race_counts
    
    # Sepsis distribution
    sepsis_count = df[df['sepsis'] == 1]['stay_id'].nunique()
    sepsis_percentage = (sepsis_count / total_stays * 100) if total_stays > 0 else 0
    results['Sepsis'] = {'Count': sepsis_count, 'Percentage': round(sepsis_percentage, 2)}
    
    # ARDS distribution
    ards_count = df[df['ards'] == 1]['stay_id'].nunique()
    ards_percentage = (ards_count / total_stays * 100) if total_stays > 0 else 0
    results['ARDS'] = {'Count': ards_count, 'Percentage': round(ards_percentage, 2)}
    
    # Gender distribution
    gender_counts = {}
    gender_columns = ['gender_F', 'gender_M'] + (['gender_Unknown'] if 'gender_Unknown' in df.columns else [])
    for col in gender_columns:
        gender_name = col.split('_')[-1].capitalize()
        count = df[df[col] == True]['stay_id'].nunique() if col == 'gender_Unknown' else df[df[col] == 1.0]['stay_id'].nunique()
        percentage = (count / total_stays * 100) if total_stays > 0 else 0
        gender_counts[gender_name] = {'Count': count, 'Percentage': round(percentage, 2)}
    results['Gender Distribution'] = gender_counts
    
    # Age distribution
    age_stats = df.groupby('stay_id')['age'].first().agg(['mean', 'std']).to_dict()
    results['Age'] = {
        'Mean': round(age_stats['mean'], 2) if not np.isnan(age_stats['mean']) else 0,
        'Std': round(age_stats['std'], 2) if not np.isnan(age_stats['std']) else 0
    }
    
    return results

def print_cohort_summary(mimic_results, eicu_results):
    """
    Print cohort description in a formatted table-like structure.
    
    Parameters:
    mimic_results (dict): Summary statistics for MIMIC-IV
    eicu_results (dict): Summary statistics for eICU
    """
    print(f"\nCohort Description for MIMIC-IV and eICU Datasets\n{'='*50}\n")
    
    # Header
    print(f"{'Characteristic':<20} {'MIMIC-IV':<25} {'eICU':<25}")
    print('-'*70)
    
    # Total Stays
    print(f"{'Total Stays':<20} {mimic_results['Total Stays']:<25} {eicu_results['Total Stays']:<25}")
    
    # Race Distribution
    print(f"\n{'Race Distribution':<20}")
    for race in mimic_results['Race Distribution'].keys():
        mimic_data = mimic_results['Race Distribution'][race]
        eicu_data = eicu_results['Race Distribution'][race]
        print(f"  {race:<18} "
              f"{mimic_data['Count']} ({mimic_data['Percentage']}%)".ljust(25) + 
              f"{eicu_data['Count']} ({eicu_data['Percentage']}%)")
    
    # Gender Distribution
    print(f"\n{'Gender Distribution':<20}")
    for gender in mimic_results['Gender Distribution'].keys():
        mimic_data = mimic_results['Gender Distribution'][gender]
        eicu_data = eicu_results['Gender Distribution'][gender]
        print(f"  {gender:<18} "
              f"{mimic_data['Count']} ({mimic_data['Percentage']}%)".ljust(25) + 
              f"{eicu_data['Count']} ({eicu_data['Percentage']}%)")
    
    # Sepsis
    print(f"\n{'Sepsis':<20} "
          f"{mimic_results['Sepsis']['Count']} ({mimic_results['Sepsis']['Percentage']}%)".ljust(25) +
          f"{eicu_results['Sepsis']['Count']} ({eicu_results['Sepsis']['Percentage']}%)")
    
    # ARDS
    print(f"{'ARDS':<20} "
          f"{mimic_results['ARDS']['Count']} ({mimic_results['ARDS']['Percentage']}%)".ljust(25) +
          f"{eicu_results['ARDS']['Count']} ({eicu_results['ARDS']['Percentage']}%)")
    
    # Age
    print(f"\n{'Age (Mean ± SD)':<20} "
          f"{mimic_results['Age']['Mean']} ± {mimic_results['Age']['Std']}".ljust(25) +
          f"{eicu_results['Age']['Mean']} ± {eicu_results['Age']['Std']}")

# Generate descriptions
mimic_results = cohort_description(mimic_iv_df, 'MIMIC-IV')
eicu_results = cohort_description(eICU_df, 'eICU')

# Print formatted summary
print_cohort_summary(mimic_results, eicu_results)

# Optional: Save results to a markdown file for thesis inclusion
def save_to_markdown(mimic_results, eicu_results, filename='cohort_description.md'):
    with open(filename, 'w') as f:
        f.write("# Cohort Description for MIMIC-IV and eICU Datasets\n\n")
        f.write("| Characteristic        | MIMIC-IV                     | eICU                         |\n")
        f.write("|-----------------------|------------------------------|------------------------------|\n")
        f.write(f"| Total Stays          | {mimic_results['Total Stays']} | {eicu_results['Total Stays']} |\n")
        f.write(f"| **Race Distribution** |                              |                              |\n")
        for race in mimic_results['Race Distribution'].keys():
            mimic_data = mimic_results['Race Distribution'][race]
            eicu_data = eicu_results['Race Distribution'][race]
            f.write(f"|   {race:<18} | "
                    f"{mimic_data['Count']} ({mimic_data['Percentage']}%) | "
                    f"{eicu_data['Count']} ({eicu_data['Percentage']}%) |\n")
        f.write(f"| **Gender Distribution** |                              |                              |\n")
        for gender in mimic_results['Gender Distribution'].keys():
            mimic_data = mimic_results['Gender Distribution'][gender]
            eicu_data = eicu_results['Gender Distribution'][gender]
            f.write(f"|   {gender:<18} | "
                    f"{mimic_data['Count']} ({mimic_data['Percentage']}%) | "
                    f"{eicu_data['Count']} ({eicu_data['Percentage']}%) |\n")
        f.write(f"| **Sepsis**           | "
                f"{mimic_results['Sepsis']['Count']} ({mimic_results['Sepsis']['Percentage']}%) | "
                f"{eicu_results['Sepsis']['Count']} ({eicu_results['Sepsis']['Percentage']}%) |\n")
        f.write(f"| **ARDS**             | "
                f"{mimic_results['ARDS']['Count']} ({mimic_results['ARDS']['Percentage']}%) | "
                f"{eicu_results['ARDS']['Count']} ({eicu_results['ARDS']['Percentage']}%) |\n")
        f.write(f"| **Age (Mean ± SD)**  | "
                f"{mimic_results['Age']['Mean']} ± {mimic_results['Age']['Std']} | "
                f"{eicu_results['Age']['Mean']} ± {eicu_results['Age']['Std']} |\n")

# Save to markdown
save_to_markdown(mimic_results, eicu_results)